In [8]:
import pandas as pd
import re
import numpy as np
from scipy import stats
from sklearn import metrics

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)

import trueskillthroughtime as ttt

In [6]:
def score_col_splitter(X, score_col):

    quarter = score_col.split("_")[0]
    
    X['Home_'+score_col] = X[score_col].apply(lambda x: x.split(" - ")[0].split(".")[-1]).astype(int)
    X['Home_'+quarter+'_Goals'] = X[score_col].apply(lambda x: x.split(" - ")[0].split(".")[0]).astype(int)
    X['Home_'+quarter+'_Behinds'] = X[score_col].apply(lambda x: x.split(" - ")[0].split(".")[1]).astype(int)
    X['Home_'+quarter+'_Shots'] = X['Home_'+quarter+'_Goals'] + X['Home_'+quarter+'_Behinds']
    X['Home_'+quarter+'_Conversion'] = X['Home_'+quarter+'_Goals'] / X['Home_'+quarter+'_Shots']
    
    X['Away_'+score_col] = X[score_col].apply(lambda x: x.split(" - ")[1].split(".")[-1]).astype(int)
    X['Away_'+quarter+'_Goals'] = X[score_col].apply(lambda x: x.split(" - ")[1].split(".")[0]).astype(int)
    X['Away_'+quarter+'_Behinds'] = X[score_col].apply(lambda x: x.split(" - ")[1].split(".")[1]).astype(int)
    X['Away_'+quarter+'_Shots'] = X['Away_'+quarter+'_Goals'] + X['Away_'+quarter+'_Behinds']
    X['Away_'+quarter+'_Conversion'] = X['Away_'+quarter+'_Goals'] / X['Away_'+quarter+'_Shots']
    
    X['Total_'+score_col] = X['Home_'+score_col] + X['Away_'+score_col]
    X['Total_'+quarter+'_Goals'] = X['Home_'+quarter+'_Goals'] + X['Away_'+quarter+'_Goals']
    X['Total_'+quarter+'_Behinds'] = X['Home_'+quarter+'_Behinds'] + X['Away_'+quarter+'_Behinds']
    X['Total_'+quarter+'_Shots'] = X['Home_'+quarter+'_Shots'] + X['Away_'+quarter+'_Shots']
    X['Total_'+quarter+'_Conversion'] = X['Total_'+quarter+'_Goals'] / X['Total_'+quarter+'_Shots']
        
    return X

In [7]:
match_summary = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/match_summary.csv")
match_summary = score_col_splitter(match_summary, "Q4_Score")
match_summary.head()

,Home_Team,Away_Team,Q4_Score,Margin,Total Game Score,Home Win,Venue,City,Date,Attendance,Temperature,Weather_Type,Year,Round_ID,Match_ID,Season,Home_Q4_Score,Home_Q4_Goals,Home_Q4_Behinds,Home_Q4_Shots,Home_Q4_Conversion,Away_Q4_Score,Away_Q4_Goals,Away_Q4_Behinds,Away_Q4_Shots,Away_Q4_Conversion,Total_Q4_Score,Total_Q4_Goals,Total_Q4_Behinds,Total_Q4_Shots,Total_Q4_Conversion
0,Brisbane Lions,Sydney,14.10.94 - 19.11.125,-31,219,0.0,Gabba,Brisbane,2021-03-20 19:45:00,0,18,MOSTLY_SUNNY,2021,202101,202101_BrisbaneLions_Sydney,NaN,94,14,10,24,0.5833,125,19,11,30,0.6333,219,33,21,54,0.6111
1,Collingwood,Western Bulldogs,7.11.53 - 10.9.69,-16,122,0.0,M.C.G.,Melbourne,2021-03-19 19:50:00,0,27,MOSTLY_SUNNY,2021,202101,202101_Collingwood_WesternBulldogs,NaN,53,7,11,18,0.3889,69,10,9,19,0.5263,122,17,20,37,0.4595
2,Essendon,Hawthorn,13.13.91 - 14.8.92,-1,183,0.0,Docklands,Melbourne,2021-03-20 19:25:00,0,26,MOSTLY_SUNNY,2021,202101,202101_Essendon_Hawthorn,NaN,91,13,13,26,0.5000,92,14,8,22,0.6364,183,27,21,48,0.5625
3,Greater Western Sydney,St Kilda,11.12.78 - 13.8.86,-8,164,0.0,Sydney Showground,Sydney,2021-03-21 15:20:00,0,22,RAIN,2021,202101,202101_GreaterWesternSydney_StKilda,NaN,78,11,12,23,0.4783,86,13,8,21,0.6190,164,24,20,44,0.5455
4,Melbourne,Fremantle,11.14.80 - 8.10.58,22,138,1.0,M.C.G.,Melbourne,2021-03-20 13:45:00,0,26,MOSTLY_SUNNY,2021,202101,202101_Melbourne_Fremantle,NaN,80,11,14,25,0.4400,58,8,10,18,0.4444,138,19,24,43,0.4419


TrueSkillThroughTime

In [13]:
brisbane = ttt.Player()
sydney = ttt.Player()
team_brisbane = [brisbane]
team_sydney = [sydney]
brisbane, sydney

(Player(Gaussian(mu=0.000, sigma=6.000), beta=1.000, gamma=0.030),
 Player(Gaussian(mu=0.000, sigma=6.000), beta=1.000, gamma=0.030))

In [21]:
match = ttt.Game([team_sydney, team_brisbane]) # Brisbane lost, Sydney Won
round(match.evidence, 3), match.likelihoods

(0.5, ([N(mu=10.781, sigma=8.958)], [N(mu=-10.781, sigma=8.958)]))

In [27]:
match.likelihoods[0][0] * sydney.prior

N(mu=3.339, sigma=4.985)

In [26]:
sydney_posterior, brisbane_posterior = match.posteriors()[0][0], match.posteriors()[1][0]
sydney_posterior, brisbane_posterior

(N(mu=3.339, sigma=4.985), N(mu=-3.339, sigma=4.985))

In [30]:
match.performance(0)

N(mu=0.000, sigma=6.083)

TrueSkill

In [31]:
import trueskill

In [76]:
brisbane = trueskill.Rating()
sydney = trueskill.Rating()
brisbane, sydney

(trueskill.Rating(mu=25.000, sigma=8.333),
 trueskill.Rating(mu=25.000, sigma=8.333))

In [77]:
print('{:.1%} chance to draw'.format(trueskill.quality_1vs1(brisbane, sydney)))

44.7% chance to draw


In [81]:
new_sydney, new_brisbane = trueskill.rate_1vs1(sydney, brisbane)
new_brisbane, new_sydney

(trueskill.Rating(mu=20.604, sigma=7.171),
 trueskill.Rating(mu=29.396, sigma=7.171))

In [82]:
import itertools
import math

def win_probability(team1, team2):
    delta_mu = team1.mu - team2.mu
    sum_sigma = (team1.sigma ** 2)+ (team2.sigma ** 2)
    denom = math.sqrt((BETA * BETA) + sum_sigma)
    ts = trueskill.global_env()
    return ts.cdf(delta_mu / denom)

In [83]:
BETA = 25/6

In [84]:
win_probability(brisbane, sydney), win_probability(sydney, brisbane)

(0.5000000150000002, 0.5000000150000002)

In [85]:
win_probability(new_brisbane, new_sydney), win_probability(new_sydney, new_brisbane)

(0.2113266125927094, 0.7886733874072906)